## Szegmentálás és tokenizálás tesztelni egy példa fasta file-on, ami a ESKAPE pathogenekből szárzmazik, ez lehet a példa kód a későbbi feladatokra.


In [1]:
%load_ext autoreload
%autoreload 2
from transformers import AutoTokenizer, AutoModel, utils
#from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings
import yaml
import pathlib
from os.path import join
import os
import sys
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

current_path = str(pathlib.Path(os.getcwd()).parent)
sys.path.append(current_path)

#import sys
#sys.path.append('../)

from config_utils import *
from sequtils import *

%load_ext autoreload
%autoreload 2

defconfig = SeqConfig()
segmentation_params = defconfig.get_segmentation_parameters()         
    
samplefasta_file = '/home/ligeti/github/prokbert/src/prokbert/data/sample_data/ESKAPE_sample.fasta'
contigs = load_contigs(samplefasta_file,IsAddHeader=True,AsDataFrame=True, adding_reverse_complement=False)
contigs['sequence_id'] = [0, 1, 2]
sequences_ls = list(contigs['sequence'])

act_segments = segment_sequence_contiguous(sequences_ls[0], segmentation_params, 1)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SEQ_CONFIG_FILE environment variable has not been set. Using default value: /home/ligeti/github/prokbert/src/prokbert/configs/sequence_processing.yaml
/home/ligeti/github/prokbert/src/prokbert
Loading sequence data into memory!
Since the fasta_files_list is a string, not list, we convert to a list.


In [ ]:
n_sequence


In [2]:
n_sequence = [ str[0:50] for str in contigs['sequence']]
test_contigs =contigs [ ['sequence_id', 'sequence']]
test_contigs['sequence'] = n_sequence
test_contigs.to_json()


/tmp/ipykernel_2519332/3597721582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_contigs['sequence'] = n_sequence


'{"sequence_id":{"0":0,"1":1,"2":2},"sequence":{"0":"TAGAAATGTCCGCGACCTTTCATACATACCACCGGTACGCCCTGGAGATG","1":"ATAATGCTAAATCGTAACCCCACTGCTTAAATGAGCCTTCTGTAAATTTC","2":"GTGACCGGGGTCAGGTTCTCGGCGGCGGCGCGCATCACGTGCTTGCCGAC"}}'

In [3]:
segment = 'TCTTTGCTAAG'
params_example = {
            'shift': 1, 
            'max_segment_length': 512, 
            'max_unknown_token_proportion': 0.2, 
            'kmer': 5, 
            'token_limit': 10
        }
params_example = defconfig.get_and_set_tokenization_params(params_example)         

result = lca_tokenize_segment(segment, params_example)

print(result)

/home/ligeti/github/prokbert/src/prokbert
([[2, 900, 515, 1022, 1004, 929, 629, 455, 3]], [['TCTTT', 'CTTTG', 'TTTGC', 'TTGCT', 'TGCTA', 'GCTAA', 'CTAAG']])


In [4]:
segment_sequences(contigs, segmentation_params, AsDataFrame = True)
 


Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame


,segment_id,sequence_id,segment_start,segment_end,segment
0,0,0,0,512,TAGAAATGTCCGCGACCTTTCATACATACCACCGGTACGCCCTGGA...
1,1,0,512,1024,CTATGAAAAAACATCTTTTAACTCTGACACTTTCCTCTATATTAGC...
2,2,0,1024,1536,TGGGTTGCAGGTTACAACTTTATGCTGGGCAGCGAGAAATTCACTC...
3,3,0,1536,2048,GCCTGCGGCTACCATCGGGAACAGCGTCGCCGGATGTCCAATCTCC...
4,4,0,2048,2560,GCTGCTGCCCTGCATCTGTAAGCACCACTTCACGCGTGGTTCTGTC...
5,5,0,2560,2775,CGTGCTGTTTCCGATTCTGGGTGTACTGTTTGCCTGGTGGAAACCG...
6,6,1,0,512,ATAATGCTAAATCGTAACCCCACTGCTTAAATGAGCCTTCTGTAAA...
7,7,1,512,1024,GTTGTATCAAATGCTTTTTGGCCAGCTAGCACTTCTTTCCATTCAA...
8,8,1,1024,1536,TTCAAATAATTCTTCACGACCAGCGTCTTTCGTAATTTGACGGCTC...
9,9,1,1536,2048,CATAACGATTTTCATCTGATTCATTTTCAGCATCAGGATCGAAATG...


## Adding the coverage based segmentation


1. lépés valamilyen dataframe-t létrehozni, amiben
cumsum, contig_lengths, sequences, contigs, ilyesmi. De nem tudom ezek között mi is a különbség



In [5]:
params = segmentation_params
params['type'] = 'random'
results = segment_sequences(contigs, params, AsDataFrame=True)
results = segment_sequences(contigs, params, AsDataFrame=False)

#pd.DataFrame(results)
results = segment_sequences(contigs, params, AsDataFrame=True)


Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Sampling 19 segments from 3 sequences.
Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Sampling 19 segments from 3 sequences.
Checking input DataFrame!
Checking input sequence_id is valid primary key in the DataFrame
Sampling 19 segments from 3 sequences.


# Tokenizáló tesztelése


In [6]:
tokenization_params = defconfig.get_and_set_tokenization_params()         
tokenization_params['shift']=2
tokenization_params['kmer']=5
print(list(results['segment'])[0][0:30])

tokens, kmers = lca_tokenize_segment(list(results['segment'])[0][0:30], tokenization_params)
#tokens, kmers = lca_tokenize_segment('ATCTGT', tokenization_params)
print(tokens)
print(kmers)
#tokens
#print(kmers)


/home/ligeti/github/prokbert/src/prokbert
ATCTGATCGGCCAGCGCGCCGAGCCCGGAA
[[2, 3], [2, 3]]
[['ATCTG', 'CTGAT', 'GATCG', 'TCGGC', 'GGCCA', 'CCAGC', 'AGCGC', 'CGCGC', 'CGCCG', 'CCGAG', 'GAGCC', 'GCCCG', 'CCGGA'], ['TCTGA', 'TGATC', 'ATCGG', 'CGGCC', 'GCCAG', 'CAGCG', 'GCGCG', 'GCGCC', 'GCCGA', 'CGAGC', 'AGCCC', 'CCCGG', 'CGGAA']]


In [ ]:
tokenization_params['vocabmap']['[CLS]']

In [ ]:
import json
# Specify the file path
file_path = "/home/ligeti/tokenizer_params.json"

# Write the dictionary to a JSON file
with open(file_path, "w") as json_file:
    json.dump(tokenization_params, json_file, indent=4)  # indent argument adds pretty formatting

tokenization_params

In [7]:
tokenization_params = defconfig.get_and_set_tokenization_params()
segments = list(results['segment'])
segment_ids = list(results['segment_id'])
batch_tokenize_segments_with_ids(segments, segment_ids, tokenization_params)


/home/ligeti/github/prokbert/src/prokbert
Tokenization of a list of segments


{0: [array([   2,  893, 3560, 1938, 3643, 2271,  878, 3498, 1689, 2647, 2382,
         1323, 1182,  619, 2462, 1642, 2459, 1629, 2407, 1422, 1578, 2202,
          603, 2399, 1389, 1445, 1670, 2570, 2076,   97,  373, 1477, 1797,
         3080,   17,   53,  197,  775, 3087,   46,  170,  665, 2646, 2380,
         1314, 1147,  480, 1907, 3519, 1774, 2987, 3744, 2675, 2493, 1765,
         2949, 3592, 2066,   57,  215,  847, 3376, 1204,  708, 2817, 3064,
         4049, 3893, 3271,  784, 3122,  188,  740, 2947, 3581, 2024, 3985,
         3637, 2248,  785, 3128,  210,  828, 3297,  888, 3539, 1854, 3305,
          917, 3653, 2309, 1030,    9,   23,   78,  299, 1183,  623, 2480,
         1716, 2753, 2808, 3027, 3901, 3303,  909, 3621, 2181,  517, 2056,
           20,   67,  253,  997, 3974, 3594, 2076,   99,  381, 1511, 1933,
         3622, 2185,  536, 2129,  310, 1225,  792, 3153,  312, 1234,  826,
         3292,  868, 3460, 1540, 2052, 4098, 4092, 4067, 3968, 3569, 1975,
         3791, 2863, 3